In [ ]:
!pip install -q pyomo

     |████████████████████████████████| 9.2 MB 4.0 MB/s 
     |████████████████████████████████| 49 kB 3.4 MB/s 


In [ ]:
# we add glpk now
!sudo apt install libglpk-dev python3.8-dev libgmp3-dev
!apt-get install -y -qq glpk-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas1 libklu1 libldl2 libmetis5
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  librbio2 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
  python3.8 python3.8-minimal
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev python3.8-venv python3.8-doc
  binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk-dev libglpk40 libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas1
  libklu1 libldl2 libmetis5 libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib librbio2 libspqr2 libsuitesparse-dev
  libsuitesparseconfig5 libumfpack5 python3.8 python3.8-dev python3.8-mini

In [ ]:
import pyomo.environ as pyo

In [ ]:
def BIM_with_revenues_minus_costs():
    model    = pyo.ConcreteModel('BIM')
    
    model.x1 = pyo.Var(within=pyo.NonNegativeReals)
    model.x2 = pyo.Var(within=pyo.NonNegativeReals)

    model.revenue       = pyo.Expression( expr = 12*model.x1  +   9*model.x2 )
    model.variable_cost = pyo.Expression( expr = 7/6*model.x1 + 5/6*model.x2 )
    model.fixed_cost    = 100

    model.profit    = pyo.Objective( sense= pyo.maximize
      , expr = model.revenue - model.variable_cost - model.fixed_cost )

    model.silicon   = pyo.Constraint(expr =    model.x1              <= 1000)
    model.germanium = pyo.Constraint(expr =                 model.x2 <= 1500)
    model.plastic   = pyo.Constraint(expr =    model.x1 +   model.x2 <= 1750)
    model.copper    = pyo.Constraint(expr =  4*model.x1 + 2*model.x2 <= 4800)

    return model

In [ ]:
def BIM_with_revenues_over_costs():
    model    = pyo.ConcreteModel('BIM')
    
    model.y1 = pyo.Var(within=pyo.NonNegativeReals)
    model.y2 = pyo.Var(within=pyo.NonNegativeReals)
    model.t  = pyo.Var(within=pyo.NonNegativeReals)

    model.revenue       = pyo.Expression( expr = 12*model.y1  +   9*model.y2 )
    model.variable_cost = pyo.Expression( expr = 7/6*model.y1 + 5/6*model.y2 )
    model.fixed_cost    = 100

    model.profit    = pyo.Objective( sense= pyo.maximize
                                   , expr = model.revenue)

    model.silicon   = pyo.Constraint(expr =    model.y1              <= 1000*model.t)
    model.germanium = pyo.Constraint(expr =                 model.y2 <= 1500*model.t)
    model.plastic   = pyo.Constraint(expr =    model.y1 +   model.y2 <= 1750*model.t)
    model.copper    = pyo.Constraint(expr =  4*model.y1 + 2*model.y2 <= 4800*model.t)
    model.frac      = pyo.Constraint(expr = model.variable_cost+model.fixed_cost*model.t == 1 )
    
    return model

In [ ]:
BIM_linear = BIM_with_revenues_minus_costs()
results = pyo.SolverFactory('glpk').solve(BIM_linear)

print('X=({:.1f},{:.1f}) value={:.3f} revenue={:.3f} cost={:.3f}'.format(
    pyo.value(BIM_linear.x1),
    pyo.value(BIM_linear.x2),
    pyo.value(BIM_linear.profit),
    pyo.value(BIM_linear.revenue),
    pyo.value(BIM_linear.variable_cost)+pyo.value(BIM_linear.fixed_cost)))

X=(650.0,1100.0) value=15925.000 revenue=17700.000 cost=1775.000


In [ ]:
BIM_fractional = BIM_with_revenues_over_costs()
results = pyo.SolverFactory('glpk').solve(BIM_fractional)
t = pyo.value(BIM_fractional.t)
print('X=({:.1f},{:.1f}) value={:.3f} revenue={:.3f} cost={:.3f}'.format(
    pyo.value(BIM_fractional.y1/t),
    pyo.value(BIM_fractional.y2/t),
    pyo.value(BIM_fractional.profit/(BIM_fractional.variable_cost+BIM_fractional.fixed_cost*t)),
    pyo.value(BIM_fractional.revenue/t),
    pyo.value(BIM_fractional.variable_cost/t)+pyo.value(BIM_fractional.fixed_cost)))

X=(250.0,1500.0) value=10.051 revenue=16500.000 cost=1641.667
